# Half-twists along the T-R arc

## Modules

In [1]:
import numpy as np
import Permutations as pm
import sympy
from numpy.random import randint as ri
from QuasiCord import *

sympy.init_printing()

import itertools

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

## Integration

In [4]:
def twist(c, hand):
    t, l, b, r = c[0], c[1], c[2], c[3]
    # ---
    delta = (t + l) - (b + r)
    alpha = 2 * t - delta  # = min(2*t-delta, 2*r+1)
    y = (t - alpha) + (r - alpha)

    f = t - r
    F = abs(2 * f - 1)
    # ---
    A = min(alpha, 2 * r + 1)
    # ---
    position = None
    td, rd = 0, 0

    # -- 1 --
    if (t - alpha >= 0) and (r - alpha >= 0):
        position = "1"
        if (t - alpha == 0) and (r - alpha == 0):
            position += "1"
            if hand == "left":
                td, rd = 1, 0
            else:
                td, rd = 1, 1
        else:
            position += "2"
            if hand == "left":
                td, rd = 2 * y + 1, 2 * (t - alpha)
            else:
                td, rd = 2 * (r - alpha) + 1, 2 * y + 1
    # -- 2 --
    elif (t - alpha) * (r - alpha) <= 0:
        position = "2"
        if r - alpha >= 0:  # --> (t - alpha < 0)
            position += "1"
            if hand == "left":
                td, rd = 2 * (r - alpha) + 1, 2 * (t - alpha)
            else:
                td, rd = F, F
        else:  # r - alpha < 0  (--> t-alpha >=0)
            position += "2"
            if hand == "left":
                td, rd = F, F
            else:
#                 print("I'm HERE!")
                td, rd = 2 * (r - A) + 1, 2 * (t - A)
    # -- 3 --
    elif (t - alpha < 0) and (r - alpha < 0):
        position = "3"
        if f > 0:
            position += "1"
            if hand == "left":
                td, rd = F, F
            else:
                if t - A >= 0:
                    position += "1"
                    td, rd = -A, 2 * (t - A)
                else:
                    position += "2"
                    if 2 * t - r >= A:  # <--> (f >= A-t)
                        position += "1"
                        # ---------------------------------------------------------------------------
                        # >>> NOTE: This is a case where the result of the twist has BR segments. <<<
                        # ---------------------------------------------------------------------------
                        td, rd = -F, 2 * (t - A)
                    else:
                        position += "2"
                        td, rd = -F, -F
        else:  # f <= 0
            position += "2"
            if hand == "left":
                position += "1"
                if A < 2 * r + 1 - t:  # <-->  alpha < 2r+1-t
                    position += "1"
                    td, rd = 2 * (r - alpha) + 1, -F
                else:
                    position += "2"
                    td, rd = -F, -F
            else:
                position += "2"
                td, rd = F, F
    # -- Exception --
    else:
        print(c, alpha, (t - alpha, r - alpha), hand, "Why??")
    # ---
    try:
        sqr = Square(c)
        # ---
        cn = (t + td, l, b, r + rd)
        nsqr = Square(cn)
        # ---
        n = cc(sqr)
        nn = cc(nsqr)
    #         if n != nn:
    #             print(
    #                 f"{c}:{n} -- {hand}_{position},{alpha},{t-alpha},{r-alpha} --> {cn}:{nn}"
    #             )
    # ---
    except Exception as ex:
        print(f"{c} -- {hand}_{position},{alpha},{t-alpha},{r-alpha} --> {cn},{ex}")
        nsqr = sqr
    # ---
    return nsqr, hand + "_" + position

In [5]:
N = 20
Max = 20
# ---
count, error = 0, 0
while count < Max*100 and error < Max:
    sqr = square_random_generator(N)
    c = sqr.code
    delta = (c[0] + c[1]) - (c[2] + c[3])
    # ---
    n = cc(sqr)
    sqrnl, pos_l = twist(c, "left")
    sqrnr, pos_r = twist(c, "right")
    nnl = cc(sqrnl)
    nnr = cc(sqrnr)
    # Output exceptions
    if n != nnl:
        print(f"{c}:{n} -- {pos_l} --> {nnl}")
        error += 1
        pass
    elif n != nnr:
        print(f"{c}:{n} -- {pos_r} --> {nnr}")
        error += 1
    else:
        pass
    #     if (c[0] - delta == 0) and (c[1]-delta)<0:
    #         print(f"{c} --(ltw)--> {twist(c, 'left').code}")
    #     if (c[1] - delta == 0) and (c[0] > c[1]):
    #         print(f"{c,n} --(ltw, {posil})--> {sqrnl.code,nnl}")
    #         print(f"{c,n} --(rtw, {posir})--> {sqrnr.code,nnr}")
    count += 1
print(count, error)

(16, 16, 8, 2) -- right_22,10,6,-8 --> (11, 16, 8, 24),T+L < B+R
(17, 10, 15, 4) -- right_311,26,-9,-22 --> (8, 10, 15, 20),T+L < B+R
(4, 15, 16, 1) -- right_311,6,-2,-5 --> (1, 15, 16, 3),T+L < B+R
(11, 9, 4, 1) -- right_22,7,4,-6 --> (8, 9, 4, 17),T+L < B+R
(18, 14, 12, 6) -- right_311,22,-4,-16 --> (5, 14, 12, 16),T+L < B+R
(15, 11, 16, 9) -- right_3121,29,-14,-20 --> (4, 11, 16, 1),T+L < B+R
(19, 2, 10, 6) -- right_311,33,-14,-27 --> (6, 2, 10, 18),T+L < B+R
(13, 10, 7, 6) -- right_311,16,-3,-10 --> (0, 10, 7, 6),T+L < B+R
(13, 14, 18, 7) -- right_3121,24,-11,-17 --> (2, 14, 18, 3),T+L < B+R
(13, 15, 7, 2) -- right_22,7,6,-5 --> (8, 15, 7, 18),T+L < B+R
(14, 17, 9, 2) -- right_22,8,6,-6 --> (9, 17, 9, 20),T+L < B+R
(13, 7, 11, 6) -- right_311,23,-10,-17 --> (0, 7, 11, 6),T+L < B+R
(9, 4, 3, 3) -- right_311,11,-2,-8 --> (2, 4, 3, 7),T+L < B+R
(14, 8, 5, 2) -- right_22,13,1,-11 --> (9, 8, 5, 20),T+L < B+R
(11, 7, 8, 1) -- right_311,13,-2,-12 --> (8, 7, 8, 17),T+L < B+R
(16, 14, 4, 2)

## Are the left and the right inverses of each other?

In [ ]:
N = 20
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    count += 1
    sqr = square_random_generator(N)
    c = sqr.code
    n = cc(sqr)

    sqr_tl, pos_tl = twist(c, "left")
    c_tl = sqr_tl.code
    n_tl = cc(sqr_tl)

    sqr_tl_tr, pos_tl_tr = twist(c_tl, "right")
    c_tl_tr = sqr_tl_tr.code
    n_tl_tr = cc(sqr_tl_tr)

    if c != c_tl_tr:  #pos_tl == 'l1':  #
        error += 1
        print(f"{c}:{n} --[{pos_tl}]-->{c_tl}:{n_tl} --[{pos_tl_tr}]-->{c_tl_tr}:{n_tl_tr}")

In [ ]:
N = 10
Max = 20
# ---
count, error = 0, 0
while count < Max*50 and error < Max:
    count += 1
    sqr = square_random_generator(N)
    c = sqr.code
    n = cc(sqr)

    sqr_tr, pos_tr = twist(c, "right")
    c_tr = sqr_tr.code
    n_tr = cc(sqr_tr)

    sqr_tr_tl, pos_tr_tl = twist(c_tr, "left")
    c_tr_tl = sqr_tr_tl.code
    n_tr_tl = cc(sqr_tr_tl)

    if c != c_tr_tl:
        error += 1
        print(f"{c}:{n} --[{pos_tr}]-->{c_tr}:{n_tr} --[{pos_tr_tl}]-->{c_tr_tl}:{n_tr_tl}")